In [184]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [185]:
def remove_noise(image):
    # Chuyển đổi sang ảnh xám
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Áp dụng Gaussian Blur ==> KIỂM TRA LẠI SIZE CỦA KERNAL 5X5 NHỚ SỐ LẺ 3X3, 5X5, 7X7
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Áp dụng Otsu's thresholding ==> KIỂM TRA LẠI SỐ 150 XEM CẦN THAY ĐỔI KO
    _, binary = cv2.threshold(blurred, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Tìm contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Tạo mask trống
    mask = np.zeros(binary.shape, np.uint8)

    for cnt in contours:
        # Tính diện tích và chu vi
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)

        # Tính độ tròn
        circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0

        # Lọc dựa trên kích thước và độ tròn ==> KIỂM TRA LẠI ĐIỀU KIỆN
        if area > 40 and circularity < 0.85:  # Điều chỉnh các giá trị này nếu cần
            cv2.drawContours(mask, [cnt], 0, (255), -1)

    # Áp dụng mask lên ảnh gốc
    result = cv2.bitwise_and(gray, gray, mask=mask)

    return result, contours

In [186]:
def non_max_suppression(boxes, overlapThresh):
  '''
  boxes: List các bounding box
  overlapThresh: Ngưỡng overlapping giữa các hình ảnh
  '''
  # Nếu không có bounding boxes thì trả về empty list
  if len(boxes)==0:
    return []
  # Nếu bounding boxes nguyên thì chuyển sang float.
  if boxes.dtype.kind == "i":
    boxes = boxes.astype("float")

  # Khởi tạo list của index được lựa chọn
  pick = []

  # Lấy ra tọa độ của các bounding boxes
  x1 = boxes[:,0]
  y1 = boxes[:,1]
  x2 = boxes[:,2]
  y2 = boxes[:,3]

  # Tính toàn diện tích của các bounding boxes và sắp xếp chúng theo thứ tự từ bottom-right, chính là tọa độ theo y của bounding box
  area = (x2 - x1 + 1) * (y2 - y1 + 1)
  idxs = np.argsort(y2)
  # Khởi tạo một vòng while loop qua các index xuất hiện trong indexes
  while len(idxs) > 0:
    # Lấy ra index cuối cùng của list các indexes và thêm giá trị index vào danh sách các indexes được lựa chọn
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)

    # Tìm cặp tọa độ lớn nhất (x, y) là điểm bắt đầu của bounding box và tọa độ nhỏ nhất (x, y) là điểm kết thúc của bounding box
    xx1 = np.maximum(x1[i], x1[idxs[:last]])
    yy1 = np.maximum(y1[i], y1[idxs[:last]])
    xx2 = np.minimum(x2[i], x2[idxs[:last]])
    yy2 = np.minimum(y2[i], y2[idxs[:last]])

    # Tính toán width và height của bounding box
    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)

    # Tính toán tỷ lệ diện tích overlap
    overlap = (w * h) / area[idxs[:last]]

    # Xóa index cuối cùng và index của bounding box mà tỷ lệ diện tích overlap > overlapThreshold
    idxs = np.delete(idxs, np.concatenate(([last],
      np.where(overlap > overlapThresh)[0])))
  # Trả ra list các index được lựa chọn
  return boxes[pick].astype("int")

In [187]:
def merge_close_bounding_boxes(boxes, threshold=50):
    # Khởi tạo danh sách các boxes mới
    merged_boxes = []

    while len(boxes) > 0:
        # Lấy một box và xóa khỏi danh sách
        box = boxes[0]
        boxes = boxes[1:]

        # Khởi tạo nhóm mới với box đầu tiên
        group = [box]

        # So sánh với các boxes còn lại
        i = 0
        while i < len(boxes):
            other_box = boxes[i]
            # Tính khoảng cách giữa các center của bounding boxes
            center_box = [(box[0] + box[2]) / 2, (box[1] + box[3]) / 2]
            center_other_box = [(other_box[0] + other_box[2]) / 2, (other_box[1] + other_box[3]) / 2]
            distance = np.linalg.norm(np.array(center_box) - np.array(center_other_box))

            # Nếu khoảng cách nhỏ hơn threshold, nhóm chúng lại
            if distance < threshold:
                group.append(other_box)
                boxes = np.delete(boxes, i, axis=0)
            else:
                i += 1

        # Tính toán bounding box mới cho nhóm
        group = np.array(group)
        x_min = np.min(group[:, 0])
        y_min = np.min(group[:, 1])
        x_max = np.max(group[:, 2])
        y_max = np.max(group[:, 3])
        merged_boxes.append([x_min, y_min, x_max, y_max])

    return merged_boxes

In [188]:
import os

In [189]:

def calculate_density(image, processed_image):
    # Tính mật độ
    foreground = np.sum(processed_image > 0)
    total_pixels = image.shape[0] * image.shape[1]
    density = foreground / total_pixels
    return density


In [190]:
DENSITY_THRESHOLD = 0.25
# CROP_SIZE_w = 960
# CROP_SIZE_h = 720


In [191]:
def process_and_save_patch(patch, output_folder, filename, suffix):
    processed_patch, _ = remove_noise(patch)
    density = calculate_density(patch, processed_patch)

    if density > DENSITY_THRESHOLD:
        save_path = os.path.join(output_folder, f'{os.path.splitext(filename)[0]}_{suffix}.png')
        cv2.imwrite(save_path, cv2.cvtColor(patch, cv2.COLOR_RGB2BGR))
        return True
    return False

In [192]:
def process_5_patch(image, output_folder, filename):
    img_h, img_w, _ = image.shape
    half_h, half_w = img_h // 2, img_w // 2

    patches = [
        ("top_left", image[:half_h, :half_w]),
        ("top_right", image[:half_h, half_w:]),
        ("bottom_left", image[half_h:, :half_w]),
        ("bottom_right", image[half_h:, half_w:]),
        ("center", image[img_h//4:3*img_h//4, img_w//4:3*img_w//4])
    ]

    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    axs = axs.flatten()

    axs[0].imshow(image)
    axs[0].set_title('Original')
    axs[0].axis('off')

    high_density_patches = []

    for i, (patch_name, patch) in enumerate(patches, start=1):
        processed_patch, _ = remove_noise(patch)
        density = calculate_density(patch, processed_patch)

        axs[i].imshow(patch)
        axs[i].set_title(f'{patch_name.capitalize()} (Density: {density:.2f})')
        axs[i].axis('off')

        if density > DENSITY_THRESHOLD:
            high_density_patches.append((patch_name, patch, density))

    plt.tight_layout()
    # plt.savefig(os.path.join(output_folder, f'{filename}_5_patches.png'))
    plt.close()

    return high_density_patches


In [193]:
def expand_box(box, image_shape, target_area):
    x, y, w, h = box
    current_area = w * h
    if current_area >= target_area:
        return box

    ratio = np.sqrt(target_area / current_area)
    new_w = int(w * ratio)
    new_h = int(h * ratio)

    center_x = x + w // 2
    center_y = y + h // 2

    new_x = max(0, center_x - new_w // 2)
    new_y = max(0, center_y - new_h // 2)

    new_x = min(new_x, image_shape[1] - new_w)
    new_y = min(new_y, image_shape[0] - new_h)

    return (new_x, new_y, new_w, new_h)

In [194]:
def process_image(folder_path, filename):
    output_folder = os.path.join(folder_path, 'crop')
    os.makedirs(output_folder, exist_ok=True)

    file_path = os.path.join(folder_path, filename)
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = image.shape
    
    if not os.path.exists(file_path):
        print(f"Lỗi: File {file_path} không tồn tại.")
        return False

    image = cv2.imread(file_path)
    if image is None:
        print(f"Lỗi: Không thể đọc file ảnh {file_path}. File có thể bị hỏng hoặc không phải là ảnh.")
        return False

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = image.shape

    min_area = (img_h * img_w) // 4

    processed_image, contours = remove_noise(image)

    all_boxes = [cv2.boundingRect(contour) for contour in contours]
    nms_boxes = non_max_suppression(np.array(all_boxes), 0.5)

    image_with_boxes = image.copy()
    for (x, y, w, h) in nms_boxes:
        cv2.rectangle(image_with_boxes, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Hiển thị ảnh với bounding boxes
    # plt.figure(figsize=(12, 8))
    # plt.imshow(image_with_boxes)
    # plt.title('Image with Bounding Boxes')
    # plt.axis('off')
    # plt.show()
    name = filename.split('.')[0]

    # # Xử lý 5 patches
    # high_density_patches = process_5_patch(image, output_folder, filename)

    large_boxes = [box for box in nms_boxes if box[2] * box[3] > min_area]

    if large_boxes:
        # Tìm bounding box lớn nhất
        largest_box = max(large_boxes, key=lambda box: box[2] * box[3])
        x, y, w, h = largest_box
        
        # Cắt hình ảnh theo bounding box lớn nhất
        largest_patch = image[y:y+h, x:x+w]
        
        # Hiển thị bounding box lớn nhất
        # plt.figure(figsize=(8, 8))
        # plt.imshow(largest_patch)
        # plt.title('Largest Bounding Box')
        # plt.axis('off')
        # plt.show()
        
        # Lưu hình ảnh với bounding box lớn nhất
        output_path = os.path.join(output_folder, f"{name}_1.png")
        plt.imsave(output_path, largest_patch)
        # print(f"Đã lưu ảnh với bounding box lớn nhất: {output_path}")
    else:
        # Trường hợp 2 và 3: Không có bounding box lớn hơn 1/4 diện tích
        high_density_patches = process_5_patch(image, output_folder, filename)
        
        if high_density_patches:
            # Có patches với density > 0.25
            fig, axs = plt.subplots(1, len(high_density_patches), figsize=(5*len(high_density_patches), 5))
            if len(high_density_patches) == 1:
                axs = [axs]
            for i, (patch_name, patch, density) in enumerate(high_density_patches):
                axs[i].imshow(patch)
                axs[i].set_title(f'{patch_name}\n(Density: {density:.2f})')
                axs[i].axis('off')
            # Lưu patch
            output_path = os.path.join(output_folder, f"{name}_{i+1}.png")
            plt.imsave(output_path, patch)
            # print(f"Đã lưu patch: {output_path}"
            # plt.tight_layout()
            # plt.show()
    
            # print(f"Hiển thị {len(high_density_patches)} patch(es) có density cao cho {filename}")
        else:
            # Không có patches với density > 0.25, mở rộng bounding box lớn nhất
            if nms_boxes.size > 0:
                largest_box = max(nms_boxes, key=lambda box: box[2] * box[3])
                expanded_box = expand_box(largest_box, (img_h, img_w), min_area)
                x, y, w, h = expanded_box
                largest_patch = image[y:y+h, x:x+w]
                
                # plt.figure(figsize=(8, 8))
                # plt.imshow(largest_patch)
                # plt.title('Largest Patch (Expanded)')
                # plt.axis('off')
                # plt.show()
                
                output_path = os.path.join(output_folder, f"{name}_1.png")
                plt.imsave(output_path, largest_patch)
                # print(f"Đã lưu patch lớn nhất (đã mở rộng): {output_path}")
            else:
                print(f"Không tìm thấy bounding box nào cho {filename}")

    return True

In [195]:
def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            print(f"Đang xử lý {filename}")
            process_image(folder_path, filename)

def process_folders(input_folders):
    for folder in input_folders:
        print(f"Đang xử lý thư mục: {folder}")
        process_folder(folder)
        print(f"Đã hoàn thành xử lý thư mục: {folder}\n")

# Danh sách các thư mục đầu vào
input_folders = [
    "dataset/NILM",
    "dataset/HSIL",
    "dataset/LSIL"


]

# Xử lý tất cả các thư mục
process_folders(input_folders)

Đang xử lý thư mục: dataset/NILM
Đang xử lý 3975.png


/var/folders/4d/mfdsg08903l0hjjm1c_fkb6h0000gn/T/ipykernel_3401/4025432461.py:43: RuntimeWarning: invalid value encountered in divide
  overlap = (w * h) / area[idxs[:last]]


Đang xử lý 8820.png
Đang xử lý 9280.png
Đang xử lý 348.png
Đang xử lý 4968.png
Đang xử lý 9294.png
Đang xử lý 1810.png
Đang xử lý 6819.png
Đang xử lý 8834.png
Đang xử lý 3961.png
Đang xử lý 6831.png
Đang xử lý 1186.png
Đang xử lý 3949.png
Đang xử lý 3791.png
Đang xử lý 4798.png
Đang xử lý 4940.png
Đang xử lý 374.png
Đang xử lý 1838.png
Đang xử lý 7291.png
Đang xử lý 360.png
Đang xử lý 7285.png
Đang xử lý 4954.png
Đang xử lý 3785.png
Đang xử lý 6825.png
Đang xử lý 8808.png
Đang xử lý 406.png
Đang xử lý 1179.png
Đang xử lý 6616.png
Đang xử lý 9525.png
Đang xử lý 4001.png
Đang xử lý 2470.png
Đang xử lý 10137.png
Đang xử lý 2316.png
Đang xử lý 5479.png
Đang xử lý 4767.png
Đang xử lý 3008.png
Đang xử lý 6170.png
Đang xử lý 9243.png
Đang xử lý 8149.png
Đang xử lý 2302.png
Đang xử lý 4773.png
Đang xử lý 4015.png
Đang xử lý 2464.png
Đang xử lý 10123.png
Đang xử lý 6602.png
Đang xử lý 9531.png
Đang xử lý 3752.png
Đang xử lý 5323.png
Đang xử lý 1145.png
Đang xử lý 9519.png
Đang xử lý 8607.png
Đa

/var/folders/4d/mfdsg08903l0hjjm1c_fkb6h0000gn/T/ipykernel_3401/2889630074.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(2, 3, figsize=(15, 10))


Đang xử lý 1421.png
Đang xử lý 7050.png
Đang xử lý 8363.png
Đang xử lý 7736.png


/var/folders/4d/mfdsg08903l0hjjm1c_fkb6h0000gn/T/ipykernel_3401/489787943.py:71: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(1, len(high_density_patches), figsize=(5*len(high_density_patches), 5))


Đang xử lý 8405.png
Đang xử lý 6428.png
Đang xử lý 1347.png
Đang xử lý 5121.png
Đang xử lý 3550.png
Đang xử lý 5135.png
Đang xử lý 2882.png
Đang xử lý 3544.png
Đang xử lý 7722.png
Đang xử lý 8411.png
Đang xử lý 1353.png
Đang xử lý 1435.png
Đang xử lý 9069.png
Đang xử lý 7044.png
Đang xử lý 8377.png
Đang xử lý 3222.png
Đang xử lý 5653.png
Đang xử lý 5914.png
Đang xử lý 980.png
Đang xử lý 9848.png
Đang xử lý 758.png
Đang xử lý 7865.png
Đang xử lý 9690.png
Đang xử lý 994.png
Đang xử lý 9684.png
Đang xử lý 5900.png
Đang xử lý 1596.png
Đang xử lý 3381.png
Đang xử lý 5928.png
Đang xử lý 5096.png
Đang xử lý 2921.png
Đang xử lý 4388.png
Đang xử lý 7681.png
Đang xử lý 9874.png
Đang xử lý 7859.png
Đang xử lý 764.png
Đang xử lý 7695.png
Đang xử lý 770.png
Đang xử lý 5082.png
Đang xử lý 2935.png
Đang xử lý 3395.png
Đang xử lý 1582.png
Đang xử lý 7118.png
Đang xử lý 9135.png
Đang xử lý 6206.png
Đang xử lý 1569.png
Đang xử lý 2060.png
Đang xử lý 4411.png
Đang xử lý 3418.png
Đang xử lý 4377.png
Đang 